##### Simulationen für MCAR mit den Top5 erklärenden Variablen (standardisiert)
In dem Notebook 'ols_prep' wird eine lineare Regression des Laptop-Preises auf alle Exogenen durchgeführt.
Anschließend werden alle Betas standardisiert und absteigend nach ihrer (absoluten) Größe sortiert. Daraus resultiert der Erklärungsgehalt der verschiedenen Exogenen. Wir nehmen die fünf Exogenen, die den größten Erklärungsgehalt haben und standardisieren sie. Sie dienen als Grundlage für die Simulationen in diesem Notebook. 
Es werden MCAR-Simulationen mit k=1000 durchgeführt, wobei jeweils nur die Top5 standardisierten Exogenen verwendet werden.
Es wird kein Jitter hinzugefügt.

In [2]:
import pandas as pd
import numpy as np
from scipy import stats
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KDTree
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

import statsmodels.api as sm
import statsmodels.formula.api as smf

rng = np.random.RandomState(69)

In [3]:
# Initialisierung von DataFrames und Data Cleaning
df = pd.read_csv('../Laptop-Preise.csv', sep=';', decimal=',')
df = df[df.extern_Schnittstellen != 2300] # Ausreißer löschen
df = df.reset_index(drop=True) # Reset Index

# Nur die 5 Spalten auswählen, die auch den größten Erklärungsgehalt haben
selected_columns = ['Preis', 'Akku_Kapazitaet', 'Arbeitsspeicher', 'Kerne', 'Marke_Apple', 'Betriebssystem_Windows']

# Create a new DataFrame with only the selected columns
df = df[selected_columns]

df_noPrice = df.drop('Preis', axis=1)
imputed_simul_knn = pd.DataFrame(columns=['Mean KNN_1', 'MSE KNN_1', 'SE KNN_1', 'Alpha KNN_1', 'Mean KNN_3', 'MSE KNN_3', 'SE KNN_3', 'Alpha KNN_3', 'Mean KNN_5', 'MSE KNN_5', 'SE KNN_5', 'Alpha KNN_5'])
imputed_stats_knn = pd.DataFrame(columns=['Mean KNN_1', 'MSE KNN_1', 'SE KNN_1', 'Alpha KNN_1', 'Mean KNN_3', 'MSE KNN_3', 'SE KNN_3', 'Alpha KNN_3', 'Mean KNN_5', 'MSE KNN_5', 'SE KNN_5', 'Alpha KNN_5'], index = np.arange(0.1, 1, 0.1))

imputed_simul_ols = pd.DataFrame(columns=['Mean', 'MSE OLS', 'SE OLS', 'Alpha OLS'])
imputed_stats_ols = pd.DataFrame(columns=['Mean', 'MSE OLS', 'SE OLS', 'Alpha OLS'], index = np.arange(0.1, 1, 0.1))

no_imputation = pd.DataFrame(columns = ['SE', 'Alpha'])

# Skalierung (Standardisierung) von df_noPrice
col_names = df_noPrice.columns
scaler = StandardScaler().fit(df_noPrice.values)
df_noPrice = scaler.transform(df_noPrice.values)
df_noPrice = pd.DataFrame(df_noPrice, columns=col_names)

# Skalierung (Standardisierung) von df mit Preis (Preis ist unverändert)
df_std = df_noPrice.copy()
df_std.insert(0, 'Preis', df['Preis'])

# Wahrer Mittelwert:
true_mean = df['Preis'].mean()

In [4]:
def contains_mean(values):
    return 1 if (true_mean > (np.mean(values) - 1.95996 * stats.sem(values))) and (true_mean < (np.mean(values) + 1.95996 * stats.sem(values))) else 0

In [5]:
# OLS Model 
# fitting the model 
# df_noPrice ist bereits standardisiert
model = sm.OLS(df['Preis'], sm.add_constant(df_noPrice)).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Preis   R-squared:                       0.765
Model:                            OLS   Adj. R-squared:                  0.764
Method:                 Least Squares   F-statistic:                     671.7
Date:                Sat, 30 Mar 2024   Prob (F-statistic):          1.96e-321
Time:                        12:19:10   Log-Likelihood:                -7590.4
No. Observations:                1038   AIC:                         1.519e+04
Df Residuals:                    1032   BIC:                         1.522e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                   1333

In [6]:
print(df_noPrice.corr().to_latex(escape=False, float_format="%.2f"))

\begin{tabular}{lrrrrr}
\toprule
{} &  Akku_Kapazitaet &  Arbeitsspeicher &  Kerne &  Marke_Apple &  Betriebssystem_Windows \\
\midrule
Akku_Kapazitaet        &             1.00 &             0.52 &   0.66 &         0.26 &                   -0.11 \\
Arbeitsspeicher        &             0.52 &             1.00 &   0.58 &         0.29 &                   -0.10 \\
Kerne                  &             0.66 &             0.58 &   1.00 &         0.16 &                   -0.07 \\
Marke_Apple            &             0.26 &             0.29 &   0.16 &         1.00 &                   -0.64 \\
Betriebssystem_Windows &            -0.11 &            -0.10 &  -0.07 &        -0.64 &                    1.00 \\
\bottomrule
\end{tabular}



C:\Users\ckopp\AppData\Local\Temp\ipykernel_18496\1598365797.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df_noPrice.corr().to_latex(escape=False, float_format="%.2f"))


In [7]:
def del_ran(df_exog, labels, chance):
    rand_array = np.random.rand(df_exog.shape[0])
    delete_entries = rand_array < chance
    keep_entries = rand_array >= chance
    
    return [df_exog[delete_entries], labels[delete_entries], df_exog[keep_entries], labels[keep_entries]]

In [8]:
def impute_ols(test_values, test_labels, train_values, train_labels):
    
    # OLS Model
    # fitting the model 
    model = sm.OLS(train_labels, sm.add_constant(train_values)).fit() 
    imputed_values = model.predict(exog = sm.add_constant(test_values, has_constant='add')).tolist()
    return [np.mean(list(train_labels) + imputed_values), np.mean((imputed_values-test_labels)**2), stats.sem(list(train_labels) + imputed_values), contains_mean(list(train_labels) + imputed_values)]

In [13]:
def impute_knn(test_values, test_labels, train_values, train_labels):
    
    tree = KDTree(train_values.values, leaf_size=5)

    imputed_values_knn_1 = []
    imputed_values_knn_3 = []
    imputed_values_knn_5 = []

    for index, entry in enumerate(test_values.values):
 
        dist, ind = tree.query([entry], k=5)
        ind = ind[0]

        current_impute_knn_1 = np.mean(train_labels.values[ind][0])
        current_impute_knn_3 = np.mean(train_labels.values[ind][:3])
        current_impute_knn_5 = np.mean(train_labels.values[ind])

        imputed_values_knn_1.append(current_impute_knn_1)
        imputed_values_knn_3.append(current_impute_knn_3)
        imputed_values_knn_5.append(current_impute_knn_5)

    mean_knn_1 = np.mean(list(train_labels.values)+imputed_values_knn_1)
    mean_knn_3 = np.mean(list(train_labels.values)+imputed_values_knn_3)
    mean_knn_5 = np.mean(list(train_labels.values)+imputed_values_knn_5)

    mse_knn_1 = np.mean((test_labels.values - imputed_values_knn_1)**2)
    mse_knn_3 = np.mean((test_labels.values - imputed_values_knn_3)**2)
    mse_knn_5 = np.mean((test_labels.values - imputed_values_knn_5)**2)

    sem_knn_1 = stats.sem(list(train_labels.values)+imputed_values_knn_1)
    sem_knn_3 = stats.sem(list(train_labels.values)+imputed_values_knn_3)
    sem_knn_5 = stats.sem(list(train_labels.values)+imputed_values_knn_5)

    alpha_knn_1 = contains_mean(list(train_labels.values)+imputed_values_knn_1)
    alpha_knn_3 = contains_mean(list(train_labels.values)+imputed_values_knn_3)
    alpha_knn_5 = contains_mean(list(train_labels.values)+imputed_values_knn_5)


    return [mean_knn_1, mse_knn_1, sem_knn_1, alpha_knn_1, mean_knn_3, mse_knn_3, sem_knn_3, alpha_knn_3, mean_knn_5, mse_knn_5, sem_knn_5, alpha_knn_5]

In [15]:
def simul_knn():
    for c in np.arange(0.1, 1, 0.1):

        imputed_simul_knn = pd.DataFrame(columns=['Mean KNN_1', 'MSE KNN_1', 'SE KNN_1', 'Alpha KNN_1', 'Mean KNN_3', 'MSE KNN_3', 'SE KNN_3', 'Alpha KNN_3', 'Mean KNN_5', 'MSE KNN_5', 'SE KNN_5', 'Alpha KNN_5'])

        for i in tqdm(range(1000)):
            temp = del_ran(df_exog = df_noPrice, labels = df['Preis'], chance = c)
            imputed_simul_knn.at[i] = impute_knn(temp[0], temp[1], temp[2], temp[3])

        return_values = []
        
        for column in imputed_simul_knn.columns:
            column_mean = np.mean(imputed_simul_knn[column].to_list())
            return_values.append(column_mean)
        # print(return_values)

        imputed_stats_knn.loc[c] =  return_values
    imputed_stats_knn

simul_knn()
imputed_stats_knn

100%|██████████| 1000/1000 [02:05<00:00,  7.99it/s]


,Mean KNN_1,MSE KNN_1,SE KNN_1,Alpha KNN_1,Mean KNN_3,MSE KNN_3,SE KNN_3,Alpha KNN_3,Mean KNN_5,MSE KNN_5,SE KNN_5,Alpha KNN_5
0.1,1328.904121,144725.958523,23.314619,1.0,1330.512545,101799.431127,23.152998,1.0,1329.303256,101453.57621,23.130188,1.0
0.2,1324.53373,148937.277874,23.359979,1.0,1327.898625,102555.773564,23.040364,1.0,1325.812603,101634.175516,23.001157,1.0
0.3,1320.614481,150109.559518,23.373354,0.996,1325.34474,102773.396164,22.911847,1.0,1321.947291,102668.821334,22.830639,1.0
0.4,1318.347609,150563.493549,23.365886,0.995,1323.373148,104724.100857,22.775051,1.0,1319.336405,104091.246262,22.640742,0.998
0.5,1317.057001,150984.801671,23.357222,0.962,1320.675393,107335.591272,22.60923,0.994,1316.602002,106478.690737,22.425439,0.993
0.6,1315.224536,153330.752355,23.265215,0.909,1318.781037,111619.441456,22.409946,0.977,1316.443462,110837.241505,22.194793,0.971
0.7,1315.451176,157534.744797,23.16521,0.875,1318.109198,118288.138527,22.171094,0.936,1319.605801,118169.839346,21.946924,0.944
0.8,1316.350486,165664.521796,23.057403,0.805,1321.411444,129788.427576,21.902743,0.881,1327.002092,131910.950046,21.495037,0.902
0.9,1319.637508,198371.575576,22.949349,0.665,1323.164278,163663.523037,21.115205,0.691,1315.873256,168944.307902,19.799125,0.599


In [16]:
def simul_ols():
    
    for c in np.arange(0.1, 1, 0.1):
        
        imputed_simul_ols = pd.DataFrame(columns=['Mean', 'MSE OLS', 'SE OLS', 'Alpha OLS'])

        for i in tqdm(range(1000)):
            
            temp = del_ran(df_exog = df_noPrice, labels = df['Preis'], chance = c)
            imputed_simul_ols.at[i] = impute_ols(temp[0], temp[1], temp[2], temp[3])

        return_values = []

        for column in imputed_simul_ols.columns:
            column_mean = np.mean(imputed_simul_ols[column].to_list())
            return_values.append(column_mean)
        # print(return_values)

        imputed_stats_ols.loc[c] =  return_values
    imputed_stats_ols

simul_ols()

  6%|▌         | 62/1000 [00:00<00:06, 151.43it/s]


ValueError: shapes (927,6) and (5,) not aligned: 6 (dim 1) != 5 (dim 0)

In [17]:
pd.concat([imputed_stats_ols, imputed_stats_knn], axis=1).to_csv('MCAR_Simulation_Top5 (neu)')
pd.concat([imputed_stats_ols, imputed_stats_knn], axis=1)

,Mean,MSE OLS,SE OLS,Alpha OLS,Mean KNN_1,MSE KNN_1,SE KNN_1,Alpha KNN_1,Mean KNN_3,MSE KNN_3,SE KNN_3,Alpha KNN_3,Mean KNN_5,MSE KNN_5,SE KNN_5,Alpha KNN_5
0.1,1333.794595,134219.669661,22.951306,1.0,1328.904121,144725.958523,23.314619,1.0,1330.512545,101799.431127,23.152998,1.0,1329.303256,101453.57621,23.130188,1.0
0.2,1333.788285,133642.519383,22.681143,1.0,1324.53373,148937.277874,23.359979,1.0,1327.898625,102555.773564,23.040364,1.0,1325.812603,101634.175516,23.001157,1.0
0.3,1333.818441,133538.395443,22.391974,1.0,1320.614481,150109.559518,23.373354,0.996,1325.34474,102773.396164,22.911847,1.0,1321.947291,102668.821334,22.830639,1.0
0.4,1334.045515,133891.566643,22.131181,1.0,1318.347609,150563.493549,23.365886,0.995,1323.373148,104724.100857,22.775051,1.0,1319.336405,104091.246262,22.640742,0.998
0.5,1333.756282,133625.111295,21.842251,1.0,1317.057001,150984.801671,23.357222,0.962,1320.675393,107335.591272,22.60923,0.994,1316.602002,106478.690737,22.425439,0.993
0.6,1333.663536,134098.651096,21.52022,0.998,1315.224536,153330.752355,23.265215,0.909,1318.781037,111619.441456,22.409946,0.977,1316.443462,110837.241505,22.194793,0.971
0.7,1332.613179,134795.991318,21.247512,0.981,1315.451176,157534.744797,23.16521,0.875,1318.109198,118288.138527,22.171094,0.936,1319.605801,118169.839346,21.946924,0.944
0.8,1333.920427,136085.76269,21.022214,0.918,1316.350486,165664.521796,23.057403,0.805,1321.411444,129788.427576,21.902743,0.881,1327.002092,131910.950046,21.495037,0.902
0.9,NaN,NaN,NaN,NaN,1319.637508,198371.575576,22.949349,0.665,1323.164278,163663.523037,21.115205,0.691,1315.873256,168944.307902,19.799125,0.599
